In [7]:
import pandas as pd
import numpy as np

In [4]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

<li> PassengerId - Numeric data, it is unique for each passenger
<li> Survived - Target data to be predicted, with incators 0 (Passenger Died) or 1 (Passenger Survived).
<li> Pclass - Numeric data, has 3 categories (1st, 2nd or 3rd
<li> Name - String data type, mostly unique for each passenger
<li> Sex - Categorical data, has 2 categories (male, female)
<li> Age - Numeric data, max age is 80, it contains few nulls (can be imputed), useful.
<li> SibSp - Numeric data, max number of siblings / spouses aboard is 8, useful.
<li> Parch - Numeric data, max number of parents / children aboard is 6, useful.
<li> Ticket - Categorical data, it has a prefix to the number, it can be useful.
<li> Fare - Numeric data, higher the class, higher is the fare (need to check), 
        max fare is 512,32 (currency is $ I guess) , useful.
<li> Cabin - Categorical data, it contains few nulls (can be imputed), it can be useful.
<li> Embarked - Categorical data, with 3 categories, C = Cherbourg, Q = Queenstown, S = Southampton, 
        it contains few nulls (can be imputed), useful.

In [33]:
#how many nulls are there?
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [34]:
train.Age.describe()

count    714.000000
mean      29.699118
std       14.526497
min        0.420000
25%       20.125000
50%       28.000000
75%       38.000000
max       80.000000
Name: Age, dtype: float64

In [19]:
train.Age.fillna(train.Age.mean(), inplace = True )
test.Age.fillna(train.Age.mean(), inplace = True )

In [21]:
#getting all nan for ages
train[train['Age'].isnull()].sample(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
584,585,0,3,"Paulner, Mr. Uscher",male,NaN,0,0,3411,8.7125,NaN,C
667,668,0,3,"Rommetvedt, Mr. Knud Paust",male,NaN,0,0,312993,7.7750,NaN,S


In [30]:
#do we have any duplicates?
train.Name.nunique() == len(train.Name.unique())
#nops =D

True

In [ ]:
Z